Code to extract twitter data.

In [ ]:
import tweepy
import time
import pandas as pd

In [ ]:
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""
bearer_token = ""

In [ ]:
client = tweepy.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret, wait_on_rate_limit=True)

In [ ]:
def get_all_tweets_keyword(keyword):

    all_tweets = []
    for response in tweepy.Paginator(client.search_recent_tweets,
                                    query = f"{keyword} -is:retweet -is:reply",
                                    user_fields = ['username', 'public_metrics'],
                                    tweet_fields = ['created_at', 'public_metrics', 'text'],
                                    expansions = ['author_id', 'referenced_tweets.id'],
                                    #  start_time = '2022-03-14T00:00:00Z',
                                    #  end_time = '2021-01-21T00:00:00Z',
                                    max_results=100):
        time.sleep(1)
        all_tweets.append(response)
    
    return all_tweets

In [ ]:
def get_all_tweets_info(all_tweets):

    result = []
    user_dict = {}
    # Loop through each response object
    for response in all_tweets:
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for user in response.includes['users']:
            user_dict[user.id] = {'username': user.username, 
                                'followers': user.public_metrics['followers_count'],
                                'tweets': user.public_metrics['tweet_count'],
                                'description': user.description,
                                'location': user.location
                                }
                                        
        for tweet in response.data:
            # For each tweet, find the author's information
            author_info = user_dict[tweet.author_id]

            try:
                reference_status = tweet.referenced_tweets[0].type
            except:
                reference_status = None
            
            # Put all of the information we want to keep in a single dictionary for each tweet
            result.append({'author_id': tweet.author_id, 
                        'username': author_info['username'],
                        'author_followers': author_info['followers'],
                        'author_tweets': author_info['tweets'],
                        'author_description': author_info['description'],
                        'author_location': author_info['location'],
                        'text': tweet.text,
                        'created_at': tweet.created_at,
                        'retweets': tweet.public_metrics['retweet_count'],
                        'replies': tweet.public_metrics['reply_count'],
                        'likes': tweet.public_metrics['like_count'],
                        'quote_count': tweet.public_metrics['quote_count'],
                        'tweet_id': tweet.id,        
                        'reference_status': reference_status
                        })
    
    return result

## The New York Times

In [ ]:
all_tweets = get_all_tweets_keyword("from: nytimes")

In [ ]:
result = get_all_tweets_info(all_tweets)

In [ ]:
df = pd.DataFrame(result)
df.to_csv("nytimes_all_tweets.csv", index = False)